# Real Estate Price Prediction - Complete Analysis

This notebook provides a comprehensive workflow for:
1. Data Ingestion - Loading all 6 CSV files
2. Data Cleaning - Fixing city typos and removing duplicates
3. Data Integration - Merging datasets
4. Exploratory Data Analysis
5. Feature Engineering
6. Model Building and Evaluation

## Setup and Imports

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 1. Data Ingestion

Load all 6 CSV files from the `01_data_raw/` folder

In [ ]:
# Define file paths
data_path = '../01_data_raw/'

# List of files to load
files = {
    'customers': 'customers.csv',
    'brokers': 'brokers.csv',
    'properties': 'properties.csv',
    'deals': 'deals.csv',
    'transactions': 'transactions.csv',
    'property_features': 'property_features.csv'
}

# Dictionary to store dataframes
dataframes = {}

# Load each CSV file
for key, filename in files.items():
    try:
        df = pd.read_csv(data_path + filename)
        dataframes[key] = df
        print(f"✓ Loaded {filename}: {df.shape[0]} rows, {df.shape[1]} columns")
    except FileNotFoundError:
        print(f"✗ File not found: {filename}")
        print(f"  Please add {filename} to the {data_path} folder")
    except Exception as e:
        print(f"✗ Error loading {filename}: {str(e)}")

print(f"\nTotal datasets loaded: {len(dataframes)}")

### Initial Data Exploration

Quick overview of each dataset

In [ ]:
# Display info for each dataset
for name, df in dataframes.items():
    print(f"\n{'='*60}")
    print(f"Dataset: {name.upper()}")
    print(f"{'='*60}")
    print(f"Shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nFirst few rows:")
    display(df.head(3))
    print(f"\nData types:")
    print(df.dtypes)
    print(f"\nMissing values:")
    print(df.isnull().sum())

## 2. Data Cleaning

### 2.1 Fix City Name Typos

Standardize city names across datasets to fix common typos and inconsistencies

In [ ]:
# Define city name mappings (common typos and variations)
city_mappings = {
    'new york': 'New York',
    'newyork': 'New York',
    'ny': 'New York',
    'nyc': 'New York',
    'los angeles': 'Los Angeles',
    'la': 'Los Angeles',
    'losangeles': 'Los Angeles',
    'san francisco': 'San Francisco',
    'sanfrancisco': 'San Francisco',
    'sf': 'San Francisco',
    'chicago': 'Chicago',
    'chicgo': 'Chicago',
    'houston': 'Houston',
    'houston ': 'Houston',
    'miami': 'Miami',
    'mimai': 'Miami'
}

def clean_city_names(df, city_column='city'):
    """
    Clean and standardize city names in a dataframe
    
    Parameters:
    - df: pandas DataFrame
    - city_column: name of the column containing city names
    
    Returns:
    - DataFrame with cleaned city names
    """
    if city_column not in df.columns:
        print(f"Warning: Column '{city_column}' not found in dataframe")
        return df
    
    df = df.copy()
    
    # Convert to lowercase for matching
    df[city_column] = df[city_column].astype(str).str.strip()
    
    # Count before cleaning
    unique_before = df[city_column].nunique()
    
    # Replace using mapping (case-insensitive)
    df[city_column] = df[city_column].apply(
        lambda x: city_mappings.get(x.lower(), x.title())
    )
    
    # Count after cleaning
    unique_after = df[city_column].nunique()
    
    print(f"City names - Before: {unique_before} unique | After: {unique_after} unique")
    
    return df

# Apply city name cleaning to relevant datasets
datasets_with_cities = ['customers', 'properties', 'brokers']

for dataset_name in datasets_with_cities:
    if dataset_name in dataframes:
        print(f"\nCleaning {dataset_name}...")
        dataframes[dataset_name] = clean_city_names(dataframes[dataset_name])
        print(f"Unique cities in {dataset_name}: {dataframes[dataset_name]['city'].unique() if 'city' in dataframes[dataset_name].columns else 'N/A'}")

### 2.2 Remove Duplicate Records

Identify and remove duplicate entries from each dataset

In [ ]:
def remove_duplicates(df, subset=None, keep='first'):
    """
    Remove duplicate rows from a dataframe
    
    Parameters:
    - df: pandas DataFrame
    - subset: column(s) to consider for identifying duplicates
    - keep: which duplicate to keep ('first', 'last', or False to remove all)
    
    Returns:
    - DataFrame without duplicates
    """
    rows_before = len(df)
    df_cleaned = df.drop_duplicates(subset=subset, keep=keep)
    rows_after = len(df_cleaned)
    duplicates_removed = rows_before - rows_after
    
    print(f"Rows before: {rows_before} | Rows after: {rows_after} | Duplicates removed: {duplicates_removed}")
    
    return df_cleaned

# Remove duplicates from each dataset
for name, df in dataframes.items():
    print(f"\nCleaning {name}...")
    dataframes[name] = remove_duplicates(df)

print("\nDuplicate removal complete!")

### 2.3 Handle Missing Values

Analyze and handle missing values in the datasets

In [ ]:
# Check missing values in each dataset
print("Missing Values Summary:")
print("="*60)

for name, df in dataframes.items():
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    
    if missing.sum() > 0:
        print(f"\n{name.upper()}:")
        missing_df = pd.DataFrame({
            'Missing Count': missing[missing > 0],
            'Percentage': missing_pct[missing > 0]
        })
        print(missing_df)
    else:
        print(f"\n{name.upper()}: No missing values")

# TODO: Implement specific strategies for handling missing values
# - Numerical: mean/median imputation or forward fill
# - Categorical: mode imputation or 'Unknown' category
# - Consider domain knowledge when choosing strategy

### 2.4 Data Type Conversions

Ensure correct data types for each column

In [ ]:
# Example: Convert date columns to datetime
date_columns = ['date', 'transaction_date', 'listing_date', 'sale_date', 'created_at', 'updated_at']

for name, df in dataframes.items():
    for col in df.columns:
        if any(date_col in col.lower() for date_col in date_columns):
            try:
                dataframes[name][col] = pd.to_datetime(df[col], errors='coerce')
                print(f"Converted {name}.{col} to datetime")
            except Exception as e:
                print(f"Could not convert {name}.{col} to datetime: {e}")

# TODO: Add more data type conversions as needed
# - Convert price/amount columns to float
# - Convert ID columns to string
# - Convert categorical columns to category dtype

## 3. Data Integration

Merge datasets to create a unified dataframe for analysis

In [ ]:
# Example integration strategy:
# 1. Start with properties as the base
# 2. Join with property_features
# 3. Join with deals
# 4. Join with transactions
# 5. Join with brokers
# 6. Join with customers

# Note: Actual merge keys depend on your data structure
# Common keys: property_id, customer_id, broker_id, deal_id, transaction_id

def integrate_datasets(dataframes):
    """
    Integrate multiple datasets into a single dataframe
    
    Parameters:
    - dataframes: dictionary of dataframes
    
    Returns:
    - Integrated dataframe
    """
    print("Starting data integration...\n")
    
    # Check if required datasets exist
    required = ['properties', 'deals']
    for req in required:
        if req not in dataframes:
            print(f"Warning: Required dataset '{req}' not found")
            return None
    
    # Start with properties as base
    integrated_df = dataframes['properties'].copy()
    print(f"Base dataset (properties): {integrated_df.shape}")
    
    # Merge with property_features (if available)
    if 'property_features' in dataframes:
        # Assuming merge on 'property_id'
        print("\nMerging with property_features...")
        # integrated_df = integrated_df.merge(dataframes['property_features'], on='property_id', how='left')
        # print(f"After property_features merge: {integrated_df.shape}")
    
    # Merge with deals
    if 'deals' in dataframes:
        print("\nMerging with deals...")
        # integrated_df = integrated_df.merge(dataframes['deals'], on='property_id', how='left')
        # print(f"After deals merge: {integrated_df.shape}")
    
    # Merge with transactions
    if 'transactions' in dataframes:
        print("\nMerging with transactions...")
        # integrated_df = integrated_df.merge(dataframes['transactions'], on='deal_id', how='left')
        # print(f"After transactions merge: {integrated_df.shape}")
    
    # Merge with brokers
    if 'brokers' in dataframes:
        print("\nMerging with brokers...")
        # integrated_df = integrated_df.merge(dataframes['brokers'], on='broker_id', how='left')
        # print(f"After brokers merge: {integrated_df.shape}")
    
    # Merge with customers
    if 'customers' in dataframes:
        print("\nMerging with customers...")
        # integrated_df = integrated_df.merge(dataframes['customers'], on='customer_id', how='left')
        # print(f"After customers merge: {integrated_df.shape}")
    
    print(f"\nFinal integrated dataset shape: {integrated_df.shape}")
    return integrated_df

# Create integrated dataset
# Uncomment when data is available
# df_integrated = integrate_datasets(dataframes)

print("\nNote: Uncomment the merge operations above once you verify the column names in your datasets")
print("Common merge keys: property_id, customer_id, broker_id, deal_id, transaction_id")

### Save Cleaned and Integrated Data

In [ ]:
# Save cleaned individual datasets
output_path = '../02_data_processed/'

for name, df in dataframes.items():
    filename = f"{output_path}{name}_cleaned.csv"
    df.to_csv(filename, index=False)
    print(f"Saved {filename}")

# Save integrated dataset
# Uncomment when integration is complete
# if 'df_integrated' in locals():
#     df_integrated.to_csv(f"{output_path}integrated_data.csv", index=False)
#     print(f"Saved {output_path}integrated_data.csv")

print("\nAll cleaned datasets saved to 02_data_processed/")

## 4. Exploratory Data Analysis (EDA)

### 4.1 Price Distribution Analysis

In [ ]:
# Assuming 'price' column exists in properties or integrated dataset
# Uncomment and modify based on your data

# # Price distribution
# plt.figure(figsize=(14, 5))
# 
# plt.subplot(1, 2, 1)
# plt.hist(df_integrated['price'], bins=50, edgecolor='black')
# plt.xlabel('Price')
# plt.ylabel('Frequency')
# plt.title('Price Distribution')
# 
# plt.subplot(1, 2, 2)
# plt.boxplot(df_integrated['price'])
# plt.ylabel('Price')
# plt.title('Price Boxplot')
# 
# plt.tight_layout()
# plt.show()
# 
# # Summary statistics
# print("Price Statistics:")
# print(df_integrated['price'].describe())

print("Uncomment the code above once you have the integrated dataset with a 'price' column")

### 4.2 Correlation Analysis

In [ ]:
# # Select numerical columns
# numerical_cols = df_integrated.select_dtypes(include=[np.number]).columns
# 
# # Calculate correlation matrix
# correlation_matrix = df_integrated[numerical_cols].corr()
# 
# # Plot heatmap
# plt.figure(figsize=(12, 10))
# sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
# plt.title('Correlation Heatmap')
# plt.tight_layout()
# plt.show()
# 
# # Show correlations with price (target variable)
# if 'price' in correlation_matrix.columns:
#     price_corr = correlation_matrix['price'].sort_values(ascending=False)
#     print("\nCorrelations with Price:")
#     print(price_corr)

print("Uncomment the code above once you have the integrated dataset")

### 4.3 Categorical Variable Analysis

In [ ]:
# # Analyze categorical variables
# categorical_cols = df_integrated.select_dtypes(include=['object', 'category']).columns
# 
# for col in categorical_cols[:5]:  # First 5 categorical columns
#     print(f"\n{col}:")
#     print(df_integrated[col].value_counts().head(10))
#     
#     # Plot if reasonable number of categories
#     if df_integrated[col].nunique() <= 10:
#         plt.figure(figsize=(10, 5))
#         df_integrated[col].value_counts().plot(kind='bar')
#         plt.title(f'Distribution of {col}')
#         plt.xlabel(col)
#         plt.ylabel('Count')
#         plt.xticks(rotation=45)
#         plt.tight_layout()
#         plt.show()

print("Uncomment the code above once you have the integrated dataset")

## 5. Feature Engineering

Create new features that might improve model performance

In [ ]:
# def create_features(df):
#     """
#     Create new features from existing columns
#     """
#     df = df.copy()
#     
#     # Example features:
#     
#     # 1. Price per square foot
#     if 'price' in df.columns and 'square_feet' in df.columns:
#         df['price_per_sqft'] = df['price'] / df['square_feet']
#     
#     # 2. Property age
#     if 'year_built' in df.columns:
#         df['property_age'] = datetime.now().year - df['year_built']
#     
#     # 3. Total rooms
#     if 'bedrooms' in df.columns and 'bathrooms' in df.columns:
#         df['total_rooms'] = df['bedrooms'] + df['bathrooms']
#     
#     # 4. Has garage (binary feature)
#     if 'garage' in df.columns:
#         df['has_garage'] = (df['garage'] > 0).astype(int)
#     
#     # 5. Days on market
#     if 'listing_date' in df.columns and 'sale_date' in df.columns:
#         df['days_on_market'] = (df['sale_date'] - df['listing_date']).dt.days
#     
#     return df
# 
# # Apply feature engineering
# df_engineered = create_features(df_integrated)
# print(f"Features after engineering: {df_engineered.shape[1]}")
# print(f"New columns: {set(df_engineered.columns) - set(df_integrated.columns)}")

print("Uncomment the code above once you have the integrated dataset")
print("Customize the feature engineering based on your specific columns")

## 6. Modeling

### 6.1 Data Preparation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# # Prepare data for modeling
# def prepare_data(df, target_column='price'):
#     """
#     Prepare data for machine learning
#     """
#     # Separate features and target
#     X = df.drop(columns=[target_column])
#     y = df[target_column]
#     
#     # Handle categorical variables
#     categorical_cols = X.select_dtypes(include=['object', 'category']).columns
#     
#     for col in categorical_cols:
#         le = LabelEncoder()
#         X[col] = le.fit_transform(X[col].astype(str))
#     
#     # Split data
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.2, random_state=42
#     )
#     
#     # Scale features
#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled = scaler.transform(X_test)
#     
#     return X_train_scaled, X_test_scaled, y_train, y_test, scaler
# 
# # Prepare the data
# X_train, X_test, y_train, y_test, scaler = prepare_data(df_engineered)
# 
# print(f"Training set size: {X_train.shape}")
# print(f"Test set size: {X_test.shape}")

print("Uncomment the code above once you have the integrated dataset")

### 6.2 Model Training and Evaluation

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# def train_and_evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
#     """
#     Train a model and evaluate its performance
#     """
#     print(f"\nTraining {model_name}...")
#     
#     # Train
#     model.fit(X_train, y_train)
#     
#     # Predict
#     y_pred_train = model.predict(X_train)
#     y_pred_test = model.predict(X_test)
#     
#     # Evaluate
#     train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
#     test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
#     train_mae = mean_absolute_error(y_train, y_pred_train)
#     test_mae = mean_absolute_error(y_test, y_pred_test)
#     train_r2 = r2_score(y_train, y_pred_train)
#     test_r2 = r2_score(y_test, y_pred_test)
#     
#     print(f"\n{model_name} Results:")
#     print(f"Train RMSE: ${train_rmse:,.2f}")
#     print(f"Test RMSE: ${test_rmse:,.2f}")
#     print(f"Train MAE: ${train_mae:,.2f}")
#     print(f"Test MAE: ${test_mae:,.2f}")
#     print(f"Train R²: {train_r2:.4f}")
#     print(f"Test R²: {test_r2:.4f}")
#     
#     return model, y_pred_test
# 
# # Dictionary to store models and predictions
# models = {}
# predictions = {}
# 
# # Linear Regression
# lr_model, lr_pred = train_and_evaluate_model(
#     LinearRegression(),
#     X_train, X_test, y_train, y_test,
#     "Linear Regression"
# )
# models['Linear Regression'] = lr_model
# predictions['Linear Regression'] = lr_pred
# 
# # Random Forest
# rf_model, rf_pred = train_and_evaluate_model(
#     RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
#     X_train, X_test, y_train, y_test,
#     "Random Forest"
# )
# models['Random Forest'] = rf_model
# predictions['Random Forest'] = rf_pred
# 
# # Gradient Boosting
# gb_model, gb_pred = train_and_evaluate_model(
#     GradientBoostingRegressor(n_estimators=100, random_state=42),
#     X_train, X_test, y_train, y_test,
#     "Gradient Boosting"
# )
# models['Gradient Boosting'] = gb_model
# predictions['Gradient Boosting'] = gb_pred

print("Uncomment the code above once you have prepared your data")

### 6.3 Model Comparison

In [ ]:
# # Compare model predictions
# plt.figure(figsize=(15, 5))
# 
# for idx, (name, pred) in enumerate(predictions.items(), 1):
#     plt.subplot(1, 3, idx)
#     plt.scatter(y_test, pred, alpha=0.5)
#     plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
#     plt.xlabel('Actual Price')
#     plt.ylabel('Predicted Price')
#     plt.title(f'{name}')
# 
# plt.tight_layout()
# plt.show()

print("Uncomment the code above once you have trained your models")

### 6.4 Feature Importance (for tree-based models)

In [ ]:
# # Plot feature importance for Random Forest
# if 'Random Forest' in models:
#     feature_importance = pd.DataFrame({
#         'feature': df_engineered.drop(columns=['price']).columns,
#         'importance': models['Random Forest'].feature_importances_
#     }).sort_values('importance', ascending=False).head(15)
#     
#     plt.figure(figsize=(10, 6))
#     plt.barh(feature_importance['feature'], feature_importance['importance'])
#     plt.xlabel('Importance')
#     plt.title('Top 15 Feature Importances (Random Forest)')
#     plt.gca().invert_yaxis()
#     plt.tight_layout()
#     plt.show()
#     
#     print("\nTop 10 Most Important Features:")
#     print(feature_importance.head(10))

print("Uncomment the code above once you have trained your Random Forest model")

### 6.5 Save Best Model

In [ ]:
import joblib

# # Save the best performing model
# best_model_name = 'Random Forest'  # Change based on your results
# best_model = models[best_model_name]
# 
# # Save model
# joblib.dump(best_model, '../02_data_processed/best_model.pkl')
# joblib.dump(scaler, '../02_data_processed/scaler.pkl')
# 
# print(f"Saved {best_model_name} model to 02_data_processed/")
# print("Model can be loaded later using: model = joblib.load('02_data_processed/best_model.pkl')")

print("Uncomment the code above once you have trained your models")

## 7. Conclusions and Next Steps

### Summary of Findings

Document your key findings here:
- Best performing model
- Most important features
- Model performance metrics
- Business insights

### Next Steps

1. **Model Improvement**:
   - Hyperparameter tuning
   - Try advanced models (XGBoost, LightGBM)
   - Ensemble methods

2. **Additional Features**:
   - External data sources (demographics, economic indicators)
   - Time-based features (seasonality)
   - Geographic features (proximity to amenities)

3. **Deployment**:
   - Create prediction API
   - Build web dashboard
   - Set up automated retraining pipeline

4. **Further Analysis**:
   - Time series analysis
   - Market segmentation
   - Price trend forecasting

In [ ]:
print("Analysis complete!")
print("\nRemember to:")
print("1. Add your CSV files to 01_data_raw/")
print("2. Uncomment and customize the code based on your actual data structure")
print("3. Adjust merge keys and column names to match your datasets")
print("4. Experiment with different models and feature engineering techniques")
print("5. Document your findings and insights")